# **Import Required Packages**

In [14]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import tensorflow as tf

# **Read the files**

In [2]:
# When using Colab, you can upload train_set.zip in the content folder and run this kernel.
!unzip -qq /content/train_set.zip 

In [3]:
# Set your directory to read the data, default is the directory in colab.
unzipped_folder_path = '/content/train_set'

In [4]:
def read_data(unzipped_folder_path):
  
  # read labels
  labels = pd.read_csv(unzipped_folder_path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(unzipped_folder_path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)


In [5]:
read_time_start=time.time()
Ori_X, Ori_Y = read_data(unzipped_folder_path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))

Read the original dataset takes 39.533 seconds


In [6]:
Ori_X.shape, Ori_Y.shape 
# should be (3000,3003) and (3000,) 
# which means 3000 number of cases 
# and 3003 numbers of pairwise distances
# of 78 fiducial points. 
# 3003 = (1+77)*78/2

((3000, 3003), (3000,))

# **Data Preprocessing For the Imbalanced Dataset & Generate New Data to Improve Learning Accuracy** 
## From the following analysis, we found that the Original Dataset is unbalanced. So we decided to generate new data for the class with smaller number of original samples. By generating new data, we not only balanced the data with equal number of samples in different class, but also create new data to help improve the learning accuracy.

* Because the number of Class 1 samples is less than the number of Class 0 samples, we decided to add more data in Class 1.
* The way we generate more data is that we randomly select two original cordinates of fiducial points in Class 1 and average them to generate new data of fiducial points and then calculate its pairwise distances and give it the label of 1. 
* It would make sense cause our models believe that the fiducial points in the same class will generate similar distribution in pairwise distances.

In [7]:
# Analyzing the data
n = Ori_Y.shape[0]
print('The number of class 0 is ' + str(n-sum(Ori_Y)))
print('The number of class 1 is ' + str(sum(Ori_Y)))
print('Only %.2f'% (sum(Ori_Y)/n*100) + '% of total dataset are class 1. ')
print('So, it is an unbalanced dataset, we need to do some data preprocessing.')
print('Here, we are using oversampling to generate more class 1 datasets.')

The number of class 0 is 2402
The number of class 1 is 598
Only 19.93% of total dataset are class 1. 
So, it is an unbalanced dataset, we need to do some data preprocessing.
Here, we are using oversampling to generate more class 1 datasets.


In [8]:
def data_preprocessing(Ori_X, Ori_Y, unzipped_folder_path):

  # data preprocessing

  distances = Ori_X
  y = Ori_Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(unzipped_folder_path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(unzipped_folder_path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y) 


In [9]:
Balanced_X, Blanced_Y = data_preprocessing(Ori_X, Ori_Y, unzipped_folder_path)

In [10]:
Balanced_X.shape, Blanced_Y.shape

((4804, 3003), (4804,))

# **Data Scaling and Train Test Split**

In [11]:
scaler = StandardScaler()
scaler.fit(Balanced_X)
distances_scale = scaler.transform(Balanced_X)

In [12]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(distances_scale, Blanced_Y, random_state=123)

In [15]:
one_hot_test=tf.one_hot(y_test,depth=2)
one_hot_train=tf.one_hot(y_train,depth=2)

# **Model 1: Logistic Regression Only**

In [16]:
start_time = time.time()
lr = LogisticRegression(random_state=123, C=0.01, max_iter=300, multi_class='multinomial' ).fit(X_train,y_train)
end_time = time.time()
print("Training the Logistics Regression uses %s seconds" % (end_time-start_time))

Training the Logistics Regression uses 14.290096759796143 seconds


In [21]:
y_fit = lr.predict(X_train)
y_fitprob = lr.decision_function(X_train)
y_pred = lr.predict(X_test)
y_predprob = lr.decision_function(X_test)
print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  0.86
Train dataset -- AUC:  0.94
Test dataset -- Accuracy:  0.85
Test dataset -- AUC:  0.92


# **Model 2: GBDT Only**
* Train with initialized GBDT

In [26]:
*start_time = time.time()
gbdt = GradientBoostingClassifier(random_state=123)
gbdt.fit(X_train,y_train)
print("Training the initialized GBDT uses %s seconds" % (time.time()-start_time))

Training the initialized GBDT uses 380.02052307128906 seconds


In [27]:
y_fit = gbdt.predict(X_train)
y_fitprob = gbdt.decision_function(X_train)
y_pred = gbdt.predict(X_test)
y_predprob = gbdt.decision_function(X_test)

print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  0.95
Train dataset -- AUC:  0.99
Test dataset -- Accuracy:  0.84
Test dataset -- AUC:  0.92


* Train with the GBDT with some choices of parameters

In [28]:
start_time = time.time()
gbdt2 = GradientBoostingClassifier(random_state=123, max_depth = 7, min_samples_split = 200,
                                  subsample = 0.85, max_features = 13, learning_rate = 0.1,
                                  n_estimators = 300)
gbdt2.fit(X_train,y_train)
print("Training the GBDT with some choices of parameters uses %s seconds" % (time.time()-start_time))

Training the GBDT with some choices of parameters uses 9.272934198379517 seconds


In [29]:
y_fit = gbdt2.predict(X_train)
y_fitprob = gbdt2.decision_function(X_train)
y_pred = gbdt2.predict(X_test)
y_predprob = gbdt2.decision_function(X_test)
print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  1.00
Train dataset -- AUC:  1.00
Test dataset -- Accuracy:  0.87
Test dataset -- AUC:  0.94


The training time is significantly reduced, and the AUC, Accuracy are higher as well. In order to find the best parameters, we need to do grid search to tune the parameters.

* Grid Search for n_estimators & learning_rate

In [ ]:
param_test1  = {  
    'n_estimators':range(100,501,100),
    'learning_rate': [0.001,0.01,0.1,0.5]
}

gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123,min_samples_split = 300,
                                  max_depth = 8 ,subsample = 0.8, max_features = 'sqrt'),
                        param_grid = param_test1, scoring='roc_auc',cv=3)
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=8,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=300,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
     

In [ ]:
gsearch1.best_params_, gsearch1.best_score_

({'learning_rate': 0.1, 'n_estimators': 500}, 0.944278946555829)

In [30]:
gsearch1_gbdt =  GradientBoostingClassifier(random_state=123, min_samples_split = 300, 
                                       learning_rate=0.1, n_estimators = 500,
                                  max_depth = 8 ,subsample = 0.8, max_features = 'sqrt')

In [31]:
start_time = time.time()
gsearch1_gbdt.fit(X_train,y_train)
print("Training the grid searched 1 model of GBDT uses %s seconds" % (time.time()-start_time))

Training the grid searched 1 model of GBDT uses 59.18402600288391 seconds


In [32]:
y_fit = gsearch1_gbdt.predict(X_train)
y_fitprob = gsearch1_gbdt.decision_function(X_train)
y_pred = gsearch1_gbdt.predict(X_test)
y_predprob = gsearch1_gbdt.decision_function(X_test)

print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  1.00
Train dataset -- AUC:  1.00
Test dataset -- Accuracy:  0.89
Test dataset -- AUC:  0.95


* Grid Search for max_depth & min_samples_split

In [ ]:
param_test2  = {  
    'max_depth':range(5,14,2),
    'min_samples_split': range(100,501,100)
}

gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123, n_estimators = 500, 
                                  subsample = 0.8, max_features = 'sqrt', learning_rate = 0.1),
                        param_grid = param_test2, scoring='roc_auc',cv=3)
gsearch2.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=500,
                                                  n_iter_no_change=None,
       

In [ ]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 11, 'min_samples_split': 200}, 0.9453475049364958)

In [33]:
gsearch2_gbdt =  GradientBoostingClassifier(random_state=123, min_samples_split = 200, 
                                       learning_rate=0.1, n_estimators = 500,
                                  max_depth = 11 ,subsample = 0.8, max_features = 'sqrt')

In [34]:
start_time = time.time()
gsearch2_gbdt.fit(X_train,y_train)
print("Training the grid searched 2 model of GBDT uses %s seconds" % (time.time()-start_time))

Training the grid searched 2 model of GBDT uses 74.36143827438354 seconds


In [35]:
y_fit = gsearch2_gbdt.predict(X_train)
y_fitprob = gsearch2_gbdt.decision_function(X_train)
y_pred = gsearch2_gbdt.predict(X_test)
y_predprob = gsearch2_gbdt.decision_function(X_test)

print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  1.00
Train dataset -- AUC:  1.00
Test dataset -- Accuracy:  0.90
Test dataset -- AUC:  0.96


* Grid Search for min_samples_leaf & subsample

In [ ]:
param_test3  = {  
     'min_samples_leaf':range(10,91,20),
     'subsample':[0.6,0.7,0.8,0.9]
}

gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123, n_estimators = 500, 
                                                               max_depth = 11, min_samples_split = 200,
                                                               max_features = 'sqrt', learning_rate = 0.1),
                        param_grid = param_test3, scoring='roc_auc',cv=3)
gsearch3.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=11,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=200,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=500,
                                                  n_iter_no_change=None,
    

In [ ]:
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_leaf': 10, 'subsample': 0.8}, 0.9462746723223064)

# After tuning the parameters of GBDT, the best GBDT model will be the following.

In [36]:
GBDT_best = GradientBoostingClassifier(random_state=123, n_estimators = 500, max_depth = 11, min_samples_split = 200,max_features = 'sqrt', min_samples_leaf=10, subsample=0.8, learning_rate = 0.1)

In [37]:
start_time = time.time()
GBDT_best.fit(X_train,y_train)
print("Training the best GBDT model uses %s seconds" % (time.time()-start_time))

Training the best GBDT model uses 73.16239047050476 seconds


In [38]:
y_fit = GBDT_best.predict(X_train)
y_fitprob = GBDT_best.decision_function(X_train)
y_pred = GBDT_best.predict(X_test)
y_predprob = GBDT_best.decision_function(X_test)

print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  1.00
Train dataset -- AUC:  1.00
Test dataset -- Accuracy:  0.90
Test dataset -- AUC:  0.96


# **Model 3: GBDT + Logistics Rregression**

In [39]:
predict_one_hot_train = tf.one_hot(y_fit, depth=2)
predict_one_hot_test = tf.one_hot(y_pred, depth=2)

In [40]:
lr_after_gbdt = LogisticRegression(penalty='l2',random_state=123)
lr_after_gbdt.fit(predict_one_hot_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
y_fit = lr_after_gbdt.predict(predict_one_hot_train)
y_fitprob = lr_after_gbdt.decision_function(predict_one_hot_train)
y_pred = lr_after_gbdt.predict(predict_one_hot_test)
y_predprob = lr_after_gbdt.decision_function(predict_one_hot_test)

print("Train dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_train, y_fit))
print("Train dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_train, y_fitprob))
print("Test dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(y_test,y_pred))
print("Test dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(y_test, y_predprob))

Train dataset -- Accuracy:  1.00
Train dataset -- AUC:  1.00
Test dataset -- Accuracy:  0.90
Test dataset -- AUC:  0.90


Combining the logistics regression and GBDT, we found it to be a weaker classifier.

## So, based on what we have in the previous parts, I believe that GBDT only is the best model in these three models, which shows AUC 0.96 and accuracy 0.90.

In [42]:
scaler = StandardScaler()
scaler.fit(Ori_X)
Ori_X_scale = scaler.transform(Ori_X)

In [43]:
y_pred = GBDT_best.predict(Ori_X_scale)
y_predprob = GBDT_best.decision_function(Ori_X_scale)

print("Original dataset -- Accuracy:  %.2f" % sklearn.metrics.accuracy_score(Ori_Y,y_pred))
print("Original dataset -- AUC:  %.2f" % sklearn.metrics.roc_auc_score(Ori_Y, y_predprob))

Original dataset -- Accuracy:  0.90
Original dataset -- AUC:  0.98


## The best GBDT-only model shows an impressive predictive AUC (0.98) in the imbalanced dataset, the original dataset and a very nice accuracy 0.90